In [1]:
# import bs4
# from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

In [3]:
from dotenv import load_dotenv
_ = load_dotenv()

In [1]:
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [2]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

num_tokens_from_string(question, "cl100k_base")

8

In [3]:
from langchain_openai import AzureOpenAIEmbeddings
import os
embd = AzureOpenAIEmbeddings(api_key = os.getenv("AZURE_OPENAI_API_KEY"), model="text-embedding-ada-002")
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

In [7]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_v2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_v2)

similarity = cosine_similarity(query_result, document_result)
print(similarity)

0.8806915835035412


In [8]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()


In [9]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50
    )

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [10]:
splits[:3]

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refi

In [11]:
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=AzureOpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [12]:
docs = retriever.invoke("What is Task Decomposition?")

In [13]:


from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


template = """Answer the question based only on the following context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context: {context}\nQuestion: {question}\n'), additional_kwargs={})])

In [14]:
llm = AzureChatOpenAI(model_name="gpt-4o", temperature=0, api_version=API_VERSION)

In [15]:
chain = prompt | llm

In [16]:
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content="Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. It involves using methods like Chain of Thought (CoT) and Tree of Thoughts to enhance model performance by instructing the model to think step by step. This process transforms big tasks into multiple manageable tasks and provides insight into the model's thinking process. Task decomposition can be achieved through simple prompting, task-specific instructions, or human inputs.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 317, 'total_tokens': 403, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4a331a0222', 'id': 'chatcmpl-CaGADyUZ3ck8k1LarZFG95eDDs2gG', 'prom